## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [11]:
df=pd.read_csv('train.csv',error_bad_lines=False,engine="python")

<ipython-input-11-a7f760701999>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('train.csv',error_bad_lines=False,engine="python")
Skipping line 1259: unexpected end of data


In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
df.shape

(1257, 5)

In [14]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
df.isnull().sum()

id          0
title      36
author    132
text        3
label       0
dtype: int64

In [16]:
###Drop Nan Values
df=df.dropna()


In [17]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [19]:
## Get the Dependent features
y=df['label']

In [20]:
X.shape

(1089, 4)

In [21]:
y.shape

(1089,)

In [22]:
import tensorflow as tf

In [23]:
tf.__version__

'2.13.0'

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [25]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [26]:
messages=X.copy()

In [27]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [28]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
1249,1249,Universal Child Season 1 Episode 24 Hosted by ...,Madeline,"Posted by Madeline | Oct 30, 2016 | 2016 , Dai..."
1251,1251,"Price for a Green Card: $500,000 Stadium Stake...",Ken Belson,"ORLANDO, Fla. — For years, sports teams hav..."
1252,1252,Michael Moore: Forget What I Said Before. Trum...,EdJenner,Go to Article \nFilmmaker Michael Moore made a...
1255,1255,The Lethal Lie of Hillary Clinton: Saving Live...,shorty,ABOVE: Image by DonkeyHotey | CC BY 2.0 “We mu...


In [48]:
messages.reset_index(inplace=True)

In [49]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
1084,1249,1249,Universal Child Season 1 Episode 24 Hosted by ...,Madeline,"Posted by Madeline | Oct 30, 2016 | 2016 , Dai..."
1085,1251,1251,"Price for a Green Card: $500,000 Stadium Stake...",Ken Belson,"ORLANDO, Fla. — For years, sports teams hav..."
1086,1252,1252,Michael Moore: Forget What I Said Before. Trum...,EdJenner,Go to Article \nFilmmaker Michael Moore made a...
1087,1255,1255,The Lethal Lie of Hillary Clinton: Saving Live...,shorty,ABOVE: Image by DonkeyHotey | CC BY 2.0 “We mu...


In [50]:
import nltk
import re
from nltk.corpus import stopwords

In [51]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [53]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [54]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [55]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2196, 1548, 3151, 3854, 4299, 4526, 1474, 4522, 871, 3382],
 [668, 2789, 2832, 1422, 1118, 328, 2556],
 [2060, 2520, 2383, 1592],
 [384, 1238, 4611, 2042, 1053, 2016],
 [3135, 1118, 3586, 4379, 3782, 953, 1118, 674, 1348, 4616],
 [3445,
  2111,
  1327,
  570,
  3296,
  4166,
  1028,
  474,
  4589,
  754,
  4752,
  4993,
  1275,
  1248,
  2556],
 [3809, 4909, 1002, 723, 321, 3144, 239, 4088, 4389, 1364, 3388],
 [4925, 3958, 2309, 4752, 2885, 1926, 4166, 2546, 4389, 1364, 3388],
 [4773, 4100, 1338, 4409, 1259, 1130, 4074, 3554, 4166, 3127],
 [2440, 524, 3072, 2220, 2820, 4004, 4907, 3110],
 [3449, 4021, 2764, 452, 59, 1126, 4179, 1906, 1566, 1913, 3647],
 [2042, 3575, 4299, 1130, 4166, 2885],
 [54, 4142, 1535, 982, 3276, 1300, 1456, 3393, 2141],
 [2924, 798, 1416, 3109, 1882, 3665, 2166, 4389, 1364, 3388],
 [3842, 1700, 2515, 2336, 230, 4389, 1364, 3388],
 [1522, 2048, 4900, 2348, 1247, 2529, 547, 4216, 4736, 30],
 [3388, 3591, 2789],
 [4769, 4884, 4566, 1400, 4166, 56, 1618, 2556],
 [

In [56]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [57]:
onehot_repr[1]

[668, 2789, 2832, 1422, 1118, 328, 2556]

### Embedding Representation

In [58]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2196 1548 3151 ...    0    0    0]
 [ 668 2789 2832 ...    0    0    0]
 [2060 2520 2383 ...    0    0    0]
 ...
 [1148  495 2690 ...    0    0    0]
 [2887  527 2789 ...    0    0    0]
 [3552 2660 1171 ...    0    0    0]]


In [59]:
embedded_docs[1]

array([ 668, 2789, 2832, 1422, 1118,  328, 2556,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [60]:
embedded_docs[0]

array([2196, 1548, 3151, 3854, 4299, 4526, 1474, 4522,  871, 3382,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [76]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [62]:
len(embedded_docs),y.shape

(1089, (1089,))

In [63]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [64]:
X_final.shape,y_final.shape

((1089, 20), (1089,))

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [77]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
12/12 [==============================] - 4s 199ms/step - loss: 0.6853 - accuracy: 0.5761 - val_loss: 0.6856 - val_accuracy: 0.5528
Epoch 2/10
12/12 [==============================] - 2s 134ms/step - loss: 0.6656 - accuracy: 0.5761 - val_loss: 0.6180 - val_accuracy: 0.5556
Epoch 3/10
12/12 [==============================] - 2s 127ms/step - loss: 0.3834 - accuracy: 0.7915 - val_loss: 0.4241 - val_accuracy: 0.8500
Epoch 4/10
12/12 [==============================] - 1s 102ms/step - loss: 0.1324 - accuracy: 0.9616 - val_loss: 0.5146 - val_accuracy: 0.8083
Epoch 5/10
12/12 [==============================] - 1s 60ms/step - loss: 0.0652 - accuracy: 0.9794 - val_loss: 0.7270 - val_accuracy: 0.7778
Epoch 6/10
12/12 [==============================] - 1s 81ms/step - loss: 0.0213 - accuracy: 0.9945 - val_loss: 0.5474 - val_accuracy: 0.8167
Epoch 7/10
12/12 [==============================] - 1s 99ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.8982 - val_accuracy: 0.7694
Epoch 8/1

### Adding Dropout

In [83]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [84]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
12/12 [==============================] - 4s 156ms/step - loss: 0.6870 - accuracy: 0.5610 - val_loss: 0.6866 - val_accuracy: 0.5528
Epoch 2/10
12/12 [==============================] - 2s 125ms/step - loss: 0.6790 - accuracy: 0.5761 - val_loss: 0.6799 - val_accuracy: 0.5528
Epoch 3/10
12/12 [==============================] - 1s 90ms/step - loss: 0.6092 - accuracy: 0.5926 - val_loss: 0.4772 - val_accuracy: 0.7250
Epoch 4/10
12/12 [==============================] - 1s 87ms/step - loss: 0.3161 - accuracy: 0.8957 - val_loss: 0.5009 - val_accuracy: 0.8083
Epoch 5/10
12/12 [==============================] - 1s 60ms/step - loss: 0.1436 - accuracy: 0.9561 - val_loss: 0.5600 - val_accuracy: 0.8083
Epoch 6/10
12/12 [==============================] - 0s 35ms/step - loss: 0.0577 - accuracy: 0.9877 - val_loss: 0.4636 - val_accuracy: 0.8361
Epoch 7/10
12/12 [==============================] - 0s 22ms/step - loss: 0.0196 - accuracy: 0.9959 - val_loss: 0.7952 - val_accuracy: 0.7917
Epoch 8/10


### Performance Metrics And Accuracy

In [85]:
y_pred=model.predict(X_test)

12/12 [==============================] - 1s 4ms/step


In [86]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [87]:
from sklearn.metrics import confusion_matrix

In [88]:
confusion_matrix(y_test,y_pred)

array([[178,  21],
       [ 57, 104]])

In [89]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7833333333333333

In [90]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.89      0.82       199
           1       0.83      0.65      0.73       161

    accuracy                           0.78       360
   macro avg       0.79      0.77      0.77       360
weighted avg       0.79      0.78      0.78       360

